In [23]:
"""
Uses cosine similarity.
Filters only books where at least 100 reviews are present.
Only where reviewers have given at least 10 reviews.
"""


import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler



In [45]:
books_df = pd.read_csv("../data/Books.csv")
users_df = pd.read_csv("../data/Users.csv")
ratings_df = pd.read_csv("../data/ratings.csv")

C:\Users\NelsonWang\AppData\Local\Temp\ipykernel_3528\3971019450.py:1: DtypeWarning: Columns (3) have mixed types. Specify dtype option on import or set low_memory=False.
  books_df = pd.read_csv("../data/Books.csv")


In [46]:
ratings_df.head()

,User-ID,ISBN,Book-Rating
0,276725,034545104X,0
1,276726,0155061224,5
2,276727,0446520802,0
3,276729,052165615X,3
4,276729,0521795028,6


In [47]:
ratings_with_book_titles = ratings_df.merge(books_df,on='ISBN')
ratings_with_book_titles.head()

,User-ID,ISBN,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-S,Image-URL-M,Image-URL-L
0,276725,034545104X,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...,http://images.amazon.com/images/P/034545104X.0...
1,276726,0155061224,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...,http://images.amazon.com/images/P/0155061224.0...
2,276727,0446520802,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...,http://images.amazon.com/images/P/0446520802.0...
3,276729,052165615X,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...,http://images.amazon.com/images/P/052165615X.0...
4,276729,0521795028,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...,http://images.amazon.com/images/P/0521795028.0...


In [48]:
ratings_with_book_titles.drop(columns=["ISBN","Image-URL-S","Image-URL-M"],axis=1,inplace=True)

In [49]:
complete_df = ratings_with_book_titles.merge(users_df.drop("Age", axis=1), on="User-ID")
complete_df.head()

,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Location
0,276725,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,"tyler, texas, usa"
1,276726,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,"seattle, washington, usa"
2,276727,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,"h, new south wales, australia"
3,276729,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,"rijeka, n/a, croatia"
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,"rijeka, n/a, croatia"


In [50]:
complete_df['Location'] = complete_df['Location'].str.split(',').str[-1].str.strip()
complete_df.head()

,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Location
0,276725,0,Flesh Tones: A Novel,M. J. Rose,2002,Ballantine Books,http://images.amazon.com/images/P/034545104X.0...,usa
1,276726,5,Rites of Passage,Judith Rae,2001,Heinle,http://images.amazon.com/images/P/0155061224.0...,usa
2,276727,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,australia
3,276729,3,Help!: Level 1,Philip Prowse,1999,Cambridge University Press,http://images.amazon.com/images/P/052165615X.0...,croatia
4,276729,6,The Amsterdam Connection : Level 4 (Cambridge ...,Sue Leather,2001,Cambridge University Press,http://images.amazon.com/images/P/0521795028.0...,croatia


In [51]:
min_user_reviewers = 10
user_ratings = complete_df.groupby('User-ID').count()
filtered_users = user_ratings[user_ratings > min_user_reviewers].index
complete_df = complete_df[complete_df['User-ID'].isin(filtered_users)]


In [52]:
min_ratings_count_threshold=100
rating_counts= complete_df.groupby('Book-Title').count()['Book-Rating']
popular_books = rating_counts[rating_counts >= min_ratings_count_threshold].index

In [53]:
final_ratings =  complete_df[complete_df['Book-Title'].isin(popular_books)]
print(f"Number of ratings: {len(final_ratings)}")
final_ratings.head()

Number of ratings: 183799


,User-ID,Book-Rating,Book-Title,Book-Author,Year-Of-Publication,Publisher,Image-URL-L,Location
2,276727,0,The Notebook,Nicholas Sparks,1996,Warner Books,http://images.amazon.com/images/P/0446520802.0...,australia
6,276744,7,A Painted House,JOHN GRISHAM,2001,Doubleday,http://images.amazon.com/images/P/038550120X.0...,usa
7,276746,0,Lightning,Dean R. Koontz,1996,Berkley Publishing Group,http://images.amazon.com/images/P/0425115801.0...,
8,276746,0,Manhattan Hunt Club,JOHN SAUL,2002,Ballantine Books,http://images.amazon.com/images/P/0449006522.0...,
9,276746,0,Dark Paradise,TAMI HOAG,1994,Bantam,http://images.amazon.com/images/P/0553561618.0...,


In [54]:
import pandas as pd
from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy

# Define the rating scale
reader = Reader(rating_scale=(0, 10))

# Load the data into Surprise's dataset format
data = Dataset.load_from_df(final_ratings[['User-ID', 'Book-Title', 'Book-Rating']], reader)

# Split the dataset into training and testing sets
train_set, test_set = train_test_split(data, test_size=0.20, random_state=42)

# Define the SVD algorithm
model = SVD()

# Train the algorithm on the training set
model.fit(train_set)

# Make predictions on the test set
predictions = model.test(test_set)

# Evaluate the model
accuracy.rmse(predictions)


RMSE: 3.8339


3.833895950814757

In [62]:
def recommend_books(user_id, n=10):
    # List all unique book titles
    all_books = final_ratings['Book-Title'].unique()

    # Remove books already rated by the user
    rated_books = final_ratings[final_ratings['User-ID'] == user_id]['Book-Title'].values
    books_to_predict = [book for book in all_books if book not in rated_books]
    # Here it makes a listing of each book, filters and finds.
    
    
    # Predict ratings for remaining books
    predictions = []
    for book in books_to_predict:
        print('trying to predict book', book)
        pred = model.predict(user_id, book)
        predictions.append((book, pred.est))

    # Sort predictions by estimated rating
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Get top N recommendations
    top_n = predictions[:n]

    return top_n

In [63]:

import random
def recommend_books_fabricated_profile(ratings:dict,  n=10):
    # List all unique book titles
    all_books = final_ratings['Book-Title'].unique()

    # Remove books already rated by the user
    # rated_books = final_ratings[final_ratings['User-ID'] == user_id]['Book-Title'].values

    rated_books = ratings.keys()

    books_to_predict = [book for book in all_books if book not in rated_books]

    # Predict ratings for remaining books
    predictions = []
    random_user_id = random.randint(2000, 1000000)
    for book in books_to_predict:
        # add history to model.
        for rated_book, rating in ratings.items():
            final_ratings = final_ratings.append({'User-ID': random_user_id, 'Book-Title': rated_book, 'Book-Rating': rating}, ignore_index=True)

        # retrain model
        data = Dataset.load_from_df(final_ratings[['User-ID', 'Book-Title', 'Book-Rating']], reader)
        model = SVD()
        model.fit(data.build_full_trainset())
        pred = model.predict(random_user_id, book)
        predictions.append((book, pred.est))

    # Sort predictions by estimated rating
    predictions.sort(key=lambda x: x[1], reverse=True)

    # Get top N recommendations
    top_n = predictions[:n]

    return top_n

In [68]:
# use closeness instead
data = Dataset.load_from_df(final_ratings[['User-ID', 'Book-Title', 'Book-Rating']], reader)
model = SVD()
model.fit(data.build_full_trainset())
product_name = "The Notebook"
model.get_neighbors(product_name, k=10)


AttributeError: 'SVD' object has no attribute 'sim'

In [64]:
def get_user_history(user_id):
    return final_ratings[final_ratings['User-ID'] == user_id]['Book-Title'].values

In [65]:

def explore_user(user_id):
    print(get_user_history(user_id))
    print("===== Recommendations ========")
    recommended_books = recommend_books(user_id)
    print(f"Top 10 recommended books for user {user_id}:")
    for i, (title, _) in enumerate(recommended_books, start=1):
        print(f"{i}. {title}")


In [66]:
user_id=276808
explore_user(user_id)

[]
===== Recommendations ========
trying to predict book The Notebook
trying to predict book A Painted House
trying to predict book Lightning
trying to predict book Manhattan Hunt Club
trying to predict book Dark Paradise
trying to predict book Night Sins
trying to predict book How Stella Got Her Groove Back
trying to predict book Waiting to Exhale
trying to predict book The Girl Who Loved Tom Gordon : A Novel
trying to predict book The Pillars of the Earth
trying to predict book The Dark Half
trying to predict book Harry Potter and the Order of the Phoenix (Book 5)
trying to predict book False Memory
trying to predict book Bridget Jones's Diary
trying to predict book A Wrinkle In Time
trying to predict book The Boy Next Door
trying to predict book Artemis Fowl (Artemis Fowl, Book 1)
trying to predict book Move to Strike
trying to predict book The Last Time They Met : A Novel
trying to predict book Back When We Were Grownups : A Novel (Ballantine Reader's Circle)
trying to predict book

In [67]:
user_ids = [276822, 276847, 276856]
for user_id in user_ids:
    explore_user(user_id)
    print("====================================="*2)

['The Boy Next Door' 'Artemis Fowl (Artemis Fowl, Book 1)']
===== Recommendations ========
trying to predict book The Notebook
trying to predict book A Painted House
trying to predict book Lightning
trying to predict book Manhattan Hunt Club
trying to predict book Dark Paradise
trying to predict book Night Sins
trying to predict book How Stella Got Her Groove Back
trying to predict book Waiting to Exhale
trying to predict book The Girl Who Loved Tom Gordon : A Novel
trying to predict book The Pillars of the Earth
trying to predict book The Dark Half
trying to predict book Harry Potter and the Order of the Phoenix (Book 5)
trying to predict book False Memory
trying to predict book Bridget Jones's Diary
trying to predict book A Wrinkle In Time
trying to predict book Move to Strike
trying to predict book The Last Time They Met : A Novel
trying to predict book Back When We Were Grownups : A Novel (Ballantine Reader's Circle)
trying to predict book Along Came a Spider (Alex Cross Novels)
tr

In [61]:
# Explore all users available interests.
all_users = final_ratings['User-ID'].unique()
for user_id in all_users[:10]:
    explore_user(user_id)
    print("====================================="*2)
    


['The Notebook']
===== Recommendations ========
Top 10 recommended books for user 276727:
1. Harry Potter and the Order of the Phoenix (Book 5)
2. Charlotte's Web (Trophy Newbery)
3. A Wrinkle In Time
4. Bastard Out of Carolina
5. Guilty Pleasures (Anita Blake Vampire Hunter (Paperback))
6. Good Omens
7. The Hobbit : The Enchanting Prelude to The Lord of the Rings
8. The Little Prince
9. Ender's Game (Ender Wiggins Saga (Paperback))
10. Stupid White Men ...and Other Sorry Excuses for the State of the Nation!
['A Painted House']
===== Recommendations ========
Top 10 recommended books for user 276744:
1. The Little Prince
2. Fast Food Nation: The Dark Side of the All-American Meal
3. The Catcher in the Rye
4. Stranger in a Strange Land (Remembering Tomorrow)
5. Big Stone Gap: A Novel (Ballantine Reader's Circle)
6. Brave New World
7. Stupid White Men ...and Other Sorry Excuses for the State of the Nation!
8. Stardust
9. A Walk in the Woods: Rediscovering America on the Appalachian Trail 

Top 10 recommended books for user 276762:
1. The Little Prince
2. Harry Potter and the Prisoner of Azkaban (Book 3)
3. Anne Frank: The Diary of a Young Girl
4. Ender's Game (Ender Wiggins Saga (Paperback))
5. Harry Potter and the Goblet of Fire (Book 4)
6. Harry Potter and the Sorcerer's Stone (Book 1)
7. Harry Potter and the Order of the Phoenix (Book 5)
8. The Perks of Being a Wallflower
9. The Hobbit : The Enchanting Prelude to The Lord of the Rings
10. Harry Potter and the Sorcerer's Stone (Harry Potter (Paperback))
['Harry Potter and the Order of the Phoenix (Book 5)' 'False Memory']
===== Recommendations ========
Top 10 recommended books for user 276788:
1. The Secret Life of Bees
2. The Hobbit : The Enchanting Prelude to The Lord of the Rings
3. The Five People You Meet in Heaven
4. The Fellowship of the Ring (The Lord of the Rings, Part 1)
5. Dune (Remembering Tomorrow)
6. A Wrinkle In Time
7. Eats, Shoots &amp; Leaves: The Zero Tolerance Approach to Punctuation
8. Suzanne's Di